In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
imdb_reviews=pd.read_csv("/content/imdb_reviews.csv")
test_reviews=pd.read_csv("/content/test_reviews.csv")

In [ ]:
imdb_reviews.head()

,Reviews,Sentiment
0,<START this film was just brilliant casting lo...,positive
1,<START big hair big boobs bad music and a gian...,negative
2,<START this has to be one of the worst films o...,negative
3,<START the <UNK> <UNK> at storytelling the tra...,positive
4,<START worst mistake of my life br br i picked...,negative


In [ ]:
word_index=pd.read_csv("/content/word_indexes.csv")

In [ ]:
word_index.head(n=10)

,Words,Indexes
0,tsukino,52009
1,nunnery,52010
2,sonja,16819
3,vani,63954
4,woods,1411
5,spiders,16118
6,hanging,2348
7,woody,2292
8,trawling,52011
9,hold's,52012


In [ ]:
word_index=dict(zip(word_index.Words,word_index.Indexes))

In [ ]:
word_index["<PAD>"]=0
word_index["<START>"]=1
word_index["<UNK>"]=2
word_index["<UNUSED>"]=3

In [ ]:
def review_encoder(text):
  arr=[word_index[word] for word in text]
  return arr

In [ ]:
train_data,train_labels=imdb_reviews['Reviews'],imdb_reviews['Sentiment']
test_data,test_labels=test_reviews['Reviews'],test_reviews['Sentiment']

In [ ]:
train_data=train_data.apply(lambda review:review.split())
test_data=test_data.apply(lambda review:review.split())

In [ ]:
train_data=train_data.apply(review_encoder)
test_data=test_data.apply(review_encoder)

In [ ]:
def encode_sentiments(sentiment):
  if sentiment=='positive':
    return 1
  else:
    return 0

train_labels=train_labels.apply(encode_sentiments)
test_labels=test_labels.apply(encode_sentiments)

In [ ]:
train_data=keras.preprocessing.sequence.pad_sequences(train_data,value=word_index["<PAD>"],padding='post',maxlen=500)
test_data=keras.preprocessing.sequence.pad_sequences(test_data,value=word_index["<PAD>"],padding='post',maxlen=500)

In [ ]:
model=keras.Sequential([keras.layers.Embedding(10000,16,input_length=500),
                        keras.layers.GlobalAveragePooling1D(),
                        keras.layers.Dense(16,activation='relu'),
                        keras.layers.Dense(1,activation='sigmoid')])

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(train_data,train_labels,epochs=50,batch_size=512,validation_data=(test_data,test_labels))

Epoch 1/50
8/8 [==============================] - 0s 42ms/step - loss: 0.0109 - accuracy: 1.0000 - val_loss: 0.0103 - val_accuracy: 1.0000
Epoch 2/50
8/8 [==============================] - 0s 33ms/step - loss: 0.0100 - accuracy: 1.0000 - val_loss: 0.0095 - val_accuracy: 1.0000
Epoch 3/50
8/8 [==============================] - 0s 33ms/step - loss: 0.0092 - accuracy: 1.0000 - val_loss: 0.0088 - val_accuracy: 1.0000
Epoch 4/50
8/8 [==============================] - 0s 35ms/step - loss: 0.0085 - accuracy: 1.0000 - val_loss: 0.0081 - val_accuracy: 1.0000
Epoch 5/50
8/8 [==============================] - 0s 36ms/step - loss: 0.0079 - accuracy: 1.0000 - val_loss: 0.0076 - val_accuracy: 1.0000
Epoch 6/50
8/8 [==============================] - 0s 34ms/step - loss: 0.0074 - accuracy: 1.0000 - val_loss: 0.0070 - val_accuracy: 1.0000
Epoch 7/50
8/8 [==============================] - 0s 36ms/step - loss: 0.0069 - accuracy: 1.0000 - val_loss: 0.0066 - val_accuracy: 1.0000
Epoch 8/50
8/8 [===========

In [ ]:
loss,accuracy=model.evaluate(test_data,test_labels)

137/137 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000


In [ ]:
index=np.random.randint(1,1000)


In [ ]:
user_review=test_reviews.loc[index]
print(user_review)

Reviews      <START quentin <UNK> partner in crime roger co...
Sentiment                                             negative
Name: 673, dtype: object


In [ ]:
user_review=test_data[index]
user_review=np.array([user_review])
if (model.predict(user_review)>0.5).astype("int32"):
  print("positive sentiment")
else:
  print("negative sentiment")

negative sentiment
